In [1]:
%%time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import NavigableString

import time
import numpy as np

from selenium.webdriver.chrome.options import Options
import threading

#--------------------------------------------------------------------------------------
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial

threadLocal = threading.local()


# Function to open web driver
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome("/usr/local/chromedriver", options=chrome_options)
    return driver


def table(url):
    
    driver = get_driver()
    
    driver.get(url)
    
    date = str(url.split('RaceDate=')[1][0:10])
    match = str(url.split('RaceNo=')[1])
    
    # Wait 10 secs so that the dynamic content has time to load.
    # Proceed to next date if page doesn't load.
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(((By.XPATH, 
                                             '//table[@class="f_tac table_bd draggable"]'))))
    except Exception as e:
        print(e, date, match, url)
        return []
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # main table
    table = soup.find('table', class_ = 'f_tac table_bd draggable')
    
    # Class, Distance, Rating table
    cdr_table = soup.find('tbody', class_ = 'f_fs13')
    
    # for cdr data
    cdr_list = []
    # for main table data
    output_list = []
    
    #horseid
    horse_id = soup.find_all(href=re.compile("HorseId"))
    horse_id_list = []
    for horse in soup.find_all(href=re.compile("HorseId")):
        horse_id = horse['href'].split('HorseId=')[1]
        horse_id_list.append([horse_id])
    
    
    # fetchin cdr table
    for cdr_row in cdr_table.find_all('tr')[1:4:2]:    # First table row(0) is blank
        # Second table row(1) is cdr, Fourth table row(3) is Prize Money
        cdr_cols = cdr_row.find_all('td')
        cdr_cols = [ele.text.strip() for ele in cdr_cols]
        
        # it stores both cdr and prize money
        cdr_list.append(cdr_cols[0])
        
    # fetching main table
    rows = table.find('tbody').find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip()
                .replace('\n','') 
                .replace(' '*20,' ')
                .replace("-", " ") for ele in cols]
        
        
        cols.append(date)
        cols.append(match)
        
        # extend cdr list to the main list
        cols.extend(cdr_list)
        output_list.append(cols)
    
    
    data = np.concatenate([output_list, horse_id_list], axis=1)
    
    driver.close()
    driver.quit()
    
    return data


# Function for multi-threading
def main():
    
    pool = ThreadPool(10)
    
    records = pool.map(table, urls)
    
    pool.close()
    pool.join()
    
    return records


if __name__ == "__main__":
    url_front = "https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate="
    
    comp = ['2007/12/15', '2008/02/09', '2008/03/09', '2008/09/21', '2008/10/19', '2008/10/26', '2008/11/08', '2008/11/16', '2008/11/30', '2009/01/01', '2009/01/04', '2009/01/07', '2009/01/28', '2009/02/11', '2009/02/15', '2009/02/22', '2009/03/01', '2009/03/07', '2009/03/15', '2009/03/22', '2009/03/28', '2009/04/05', '2009/04/13', '2009/04/22', '2009/04/26', '2009/05/01', '2009/05/06', '2009/05/17', '2009/05/21', '2009/05/31', '2009/06/03', '2009/06/07', '2009/06/13', '2009/06/21', '2009/06/24', '2009/06/28', '2009/07/01', '2009/09/13', '2009/09/20', '2009/10/01', '2009/10/04', '2009/10/07', '2009/10/11', '2009/10/17', '2009/10/21', '2009/10/25', '2009/11/01', '2009/11/04', '2009/11/07', '2009/11/15', '2009/11/18', '2009/11/22', '2009/11/25', '2009/11/29', '2009/12/02', '2009/12/06', '2009/12/09', '2009/12/13', '2009/12/16', '2009/12/19', '2009/12/23', '2009/12/27', '2010/01/01', '2010/01/06', '2010/01/10', '2010/01/13', '2010/01/16', '2010/01/20', '2010/01/31', '2010/02/03', '2010/02/07', '2010/02/10', '2010/02/16', '2010/02/21', '2010/02/24', '2010/02/28', '2010/03/03', '2010/03/06', '2010/03/10', '2010/03/14', '2010/03/21', '2010/03/24', '2010/03/28', '2010/03/31', '2010/04/04', '2010/04/10', '2010/04/18', '2010/04/21', '2010/04/25', '2010/05/01', '2010/05/05', '2010/05/08', '2010/05/12', '2010/05/16', '2010/05/19', '2010/05/23', '2010/05/26', '2010/05/30', '2010/06/02', '2010/06/06', '2010/06/09', '2010/06/12', '2010/06/16', '2010/06/20', '2010/06/27', '2010/07/01', '2010/07/04', '2010/07/07', '2010/07/11', '2010/07/14', '2010/09/05', '2010/09/08', '2010/09/12', '2010/09/15', '2010/09/18', '2010/09/23', '2010/09/26', '2010/10/01', '2010/10/06', '2010/10/10', '2010/10/13', '2010/10/17', '2010/10/20', '2010/10/24', '2010/10/31', '2010/11/03', '2010/11/06', '2010/11/10', '2010/11/14', '2010/11/17', '2010/11/21', '2010/11/24', '2010/11/28', '2010/12/01', '2010/12/04', '2010/12/08', '2010/12/12', '2010/12/15', '2010/12/19', '2010/12/23', '2010/12/27', '2011/01/01', '2011/01/05', '2011/01/09', '2011/01/12', '2011/01/16', '2011/01/19', '2011/01/23', '2011/01/26', '2011/01/30', '2011/02/05', '2011/02/09', '2011/02/12', '2011/02/16', '2011/02/20', '2011/02/23', '2011/02/27', '2011/03/02', '2011/03/06', '2011/03/09', '2011/03/12', '2011/03/16', '2011/03/20', '2011/03/23', '2011/03/27', '2011/03/30', '2011/04/03', '2011/04/06', '2011/04/09', '2011/04/13', '2011/04/17', '2011/04/20', '2011/04/25', '2011/05/01', '2011/05/04', '2011/05/07', '2011/05/10', '2011/05/15', '2011/05/18', '2011/05/21', '2011/05/25', '2011/05/29', '2011/06/01', '2011/06/05', '2011/06/08', '2011/06/11', '2011/06/15', '2011/06/19', '2011/06/22', '2011/06/26', '2011/07/01', '2011/07/06', '2011/07/10', '2011/09/11', '2011/09/14', '2011/09/18', '2011/09/21', '2011/09/25', '2011/09/28', '2011/10/01', '2011/10/06', '2011/10/09', '2011/10/12', '2011/10/16', '2011/10/19', '2011/10/23', '2011/10/26', '2011/10/30', '2011/11/05', '2011/11/09', '2011/11/13', '2011/11/16', '2011/11/20', '2011/11/23', '2011/11/27', '2011/11/30', '2011/12/04', '2011/12/07', '2011/12/11', '2011/12/14', '2011/12/17', '2011/12/21', '2011/12/27', '2012/01/01', '2012/01/04', '2012/01/08', '2012/01/11', '2012/01/15', '2012/01/18', '2012/01/21', '2012/01/25', '2012/01/29', '2012/02/01', '2012/02/05', '2012/02/08', '2012/02/11', '2012/02/15', '2012/02/19', '2012/02/22', '2012/02/26', '2012/02/29', '2012/03/04', '2012/03/07', '2012/03/10', '2012/03/14', '2012/03/18', '2012/03/21', '2012/03/25', '2012/03/28', '2012/04/01', '2012/04/09', '2012/04/15', '2012/04/18', '2012/04/21', '2012/04/25', '2012/04/29', '2012/05/02', '2012/05/06', '2012/05/09', '2012/05/12', '2012/05/16', '2012/05/19', '2012/05/27', '2012/05/30', '2012/06/03', '2012/06/06', '2012/06/09', '2012/06/13', '2012/06/17', '2012/06/20', '2012/06/24', '2012/07/01', '2012/07/04', '2012/07/08', '2012/07/11', '2012/07/15', '2012/09/08', '2012/09/12', '2012/09/16', '2012/09/19', '2012/09/23', '2012/09/26', '2012/10/01', '2012/10/06', '2012/10/10', '2012/10/14', '2012/10/17', '2012/10/21', '2012/10/24', '2012/10/28', '2012/11/04', '2012/11/07', '2012/11/10', '2012/11/14', '2012/11/18', '2012/11/21', '2012/11/25', '2012/11/28', '2012/12/02', '2012/12/05', '2012/12/09', '2012/12/12', '2012/12/16', '2012/12/19', '2012/12/22', '2012/12/28', '2013/01/01', '2013/01/06', '2013/01/09', '2013/01/12', '2013/01/16', '2013/01/20', '2013/01/23', '2013/01/27', '2013/01/30', '2013/02/02', '2013/02/06', '2013/02/12', '2013/02/17', '2013/02/20', '2013/02/24', '2013/02/27', '2013/03/02', '2013/03/06', '2013/03/10', '2013/03/13', '2013/03/17', '2013/03/20', '2013/03/24', '2013/03/27', '2013/04/01', '2013/04/07', '2013/04/10', '2013/04/14', '2013/04/17', '2013/04/20', '2013/04/24', '2013/04/28', '2013/05/01', '2013/05/05', '2013/05/08', '2013/05/11', '2013/05/15', '2013/05/18', '2013/05/22', '2013/05/26', '2013/05/29', '2013/06/02', '2013/06/05', '2013/06/08', '2013/06/12', '2013/06/16', '2013/06/19', '2013/06/23', '2013/06/26', '2013/07/01', '2013/07/04', '2013/07/07', '2013/07/10', '2013/09/08', '2013/09/11', '2013/09/15', '2013/09/17', '2013/09/25', '2013/10/01', '2013/10/06', '2013/10/09', '2013/10/12', '2013/10/16', '2013/10/20', '2013/10/23', '2013/10/27', '2013/10/30', '2013/11/03', '2013/11/06', '2013/11/09', '2013/11/13', '2013/11/17', '2013/11/20', '2013/11/24', '2013/11/27', '2013/12/01', '2013/12/04', '2013/12/08', '2013/12/11', '2013/12/15', '2013/12/18', '2013/12/21', '2013/12/26', '2013/12/29', '2014/01/01', '2014/01/05', '2014/01/08', '2014/01/11', '2014/01/15', '2014/01/19', '2014/01/22', '2014/01/26', '2014/02/02', '2014/02/05', '2014/02/08', '2014/02/12', '2014/02/16', '2014/02/19', '2014/02/23', '2014/02/26', '2014/03/01', '2014/03/05', '2014/03/09', '2014/03/12', '2014/03/16', '2014/03/19', '2014/03/23', '2014/03/26', '2014/03/30', '2014/04/02', '2014/04/06', '2014/04/09', '2014/04/13', '2014/04/16', '2014/04/21', '2014/04/27', '2014/04/30', '2014/05/04', '2014/05/07', '2014/05/10', '2014/05/14', '2014/05/17', '2014/05/21', '2014/05/25', '2014/05/28', '2014/06/01', '2014/06/05', '2014/06/08', '2014/06/11', '2014/06/15', '2014/06/18', '2014/06/22', '2014/06/25', '2014/06/28', '2014/07/01', '2014/07/06', '2014/09/14', '2014/09/17', '2014/09/21', '2014/09/24', '2014/09/27', '2014/10/01', '2014/10/05', '2014/10/08', '2014/10/12', '2014/10/15', '2014/10/19', '2014/10/22', '2014/10/26', '2014/10/29', '2014/11/02', '2014/11/09', '2014/11/12', '2014/11/15', '2014/11/19', '2014/11/23', '2014/11/26', '2014/11/30', '2014/12/03', '2014/12/07', '2014/12/10', '2014/12/14', '2014/12/17', '2014/12/20', '2014/12/28']
    
    urls = []
    
    for date in comp:
        for match in range(1,13):
            # create a list of url
            urls.append(url_front + date + '&RaceNo=' + str(match))
    

    
    result = main()
    
    # Flaten a list of list
    result = [item for items in result for item in items]
    
    write_to_csv = pd.DataFrame(result, 
                                columns = ['Plc', 'Horse No', 'Horse', 'Jockey', 'Trainer', 'ActualWt',
                                           'Declar_HorseWt', 'Draw', 'LBW', 'RunningPosition', 'FinishTime', 
                                           'Win_Odds', 'date', 'match', 'cdr', 'prize_money', 'horseid'])                                    
    
    

Message: 
 2010/03/28 11 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/03/28&RaceNo=11
Message: 
 2010/05/19 9 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/05/19&RaceNo=9
Message: 
 2009/01/07 11 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2009/01/07&RaceNo=11
Message: 
 2009/04/22 9 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2009/04/22&RaceNo=9
Message: 
 2010/03/28 12 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/03/28&RaceNo=12
Message: 
 2010/05/19 10 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/05/19&RaceNo=10
Message: 
 2009/01/07 12 https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2009/01/07&RaceNo=12
Message: 
 2009/04/22 10 https://racing.hkjc.com/racing/information/english/Racing/Loca

In [2]:
write_to_csv

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
0,1,11,ARISTOCRACY(CG070),Z Purton,C S Shum,120,1104,11,,1:41.20,16,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2005_G070,None
1,2,4,COULDN'T CARE LESS(CD009),S Dye,A Schutz,126,1080,1,NOSE,1:41.20,4.2,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2003_D009,None
2,3,5,AS YOU WISH(CE290),A Delpech,B K Ng,125,1048,7,N,1:41.30,12,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2004_E290,None
3,4,7,NOBLE ZOOM(CC298),P H Lo,A Lee,118,1084,5,1 1/2,1:41.40,11,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2002_C298,None
4,5,12,MASTER YING(CB249),E Saint Martin,T K Ng,119,1166,4,1 1/2,1:41.40,9.3,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2001_B249,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56458,11,11,BULLISH BOY(P008),M L Yeung,A S Cruz,115,1093,6,6 3/4,2 2 2 ...,1:23.03,99,2014/12/28,11,Class 2 - 1400M - (100-80),"HK$ 1,500,000",HK_2012_P008
56459,12,13,OUR FOLKS(P400),M Chadwick,C W Chang,117,1153,12,7,8 9 10 ...,1:23.04,34,2014/12/28,11,Class 2 - 1400M - (100-80),"HK$ 1,500,000",HK_2012_P400
56460,13,6,BEAR HERO(M220),N Callan,D E Ferraris,122,1127,10,8 3/4,12 12 12 ...,1:23.32,24,2014/12/28,11,Class 2 - 1400M - (100-80),"HK$ 1,500,000",HK_2010_M220
56461,14,2,MY NAME IS BOND(M287),C Y Ho,C Fownes,124,1094,13,9 1/4,14 14 13 ...,1:23.42,99,2014/12/28,11,Class 2 - 1400M - (100-80),"HK$ 1,500,000",HK_2010_M287


In [3]:
# the path you want to save the result
filepath = "clean/race_result_comp.csv"

write_to_csv.to_csv(filepath, index=False)